In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt

/Users/yuzu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!pip3 install --upgrade gradio

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [2]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for Vulcan Fossil Fuel CO₂ Emissions, Version 4. 
collection_ffco2 = "odiac-ffco2-monthgrid-v2023"
# Name of the collection for MiCASA Land Carbon Flux
collection_landcarbon = "micasa-carbonflux-daygrid-v1"

In [3]:
collection_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2}").json()
collection_landcarbon = requests.get(f"{STAC_API_URL}/collections/{collection_landcarbon}").json()

In [6]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [7]:
num_items_ffco2 = get_item_count(collection_ffco2["id"])

items_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2['id']}/items?limit={num_items_ffco2}").json()["features"]

In [8]:
# num_items_landcarbon = get_item_count(collection_landcarbon["id"])
num_items_landcarbon = 800

items_landcarbon = requests.get(f"{STAC_API_URL}/collections/{collection_landcarbon['id']}/items?limit={num_items_landcarbon}").json()["features"]

In [9]:
items_ffco2 = {item["properties"]["start_datetime"][:7]: item for item in items_ffco2}
items_landcarbon = {item["properties"]["datetime"][:10]: item for item in items_landcarbon}

asset_ffco2 = "co2-emissions"
asset_landcarbon = "rh"

In [10]:
rescale_ffco2 = {"max":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["max"], "min":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["min"]}
rescale_landcarbon = {"max":items_landcarbon[list(items_landcarbon.keys())[0]]["assets"][asset_landcarbon]["raster:bands"][0]["histogram"]["max"], "min":items_landcarbon[list(items_landcarbon.keys())[0]]["assets"][asset_landcarbon]["raster:bands"][0]["histogram"]["min"]}

In [11]:
color_map = "spectral_r"

def year_map(year):
    year = str(year)
    date = year + "-12"
    _ffco2_tile = requests.get(
        f"{RASTER_API_URL}/collections/{items_ffco2[date]['collection']}/items/{items_ffco2[date]['id']}/tilejson.json?"
        f"&assets={asset_ffco2}"
        f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
        f"&rescale={rescale_ffco2['min']},{rescale_ffco2['max']}", 
    ).json()
    
    date= year + "-12-31"
    _landcarbon_tile = requests.get(
        f"{RASTER_API_URL}/collections/{items_landcarbon[date]['collection']}/items/{items_landcarbon[date]['id']}/tilejson.json?"
        f"&assets={asset_landcarbon}"
        f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
        f"&rescale={rescale_landcarbon['min']},{rescale_landcarbon['max']}", 
    ).json()

    map_ = folium.plugins.DualMap(location=(34, -118), zoom_start=6)
    map_layer_ffco2 = TileLayer(
        tiles=_ffco2_tile["tiles"][0],
        attr="GHG",
        name=f'{year} Total CO2 Fossil Fuel Emissions',
        overlay=True,
        opacity=0.8,
    )
    map_layer_ffco2.add_to(map_.m1)
    
    map_layer_landcarbon = TileLayer(
        tiles=_landcarbon_tile["tiles"][0],
        attr="GHG",
        name=f'{year} RH Level',
        overlay=True,
        opacity=0.8,
    )
    map_layer_landcarbon.add_to(map_.m2)
    
    return map_._repr_html_()

In [12]:
year_map("2022")

'<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><span style="color:#565656">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;html&gt;\n&lt;head&gt;\n    \n    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n    \n        &lt;script&gt;\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        &lt;/script&gt;\n    \n    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://code.jquery.com/jquery-3.7.1.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js&quot;&gt;&lt;/sc

In [16]:
import gradio as gr

# 画像のパス
image_path = "./../images/nasa_space_apps.png"

# カスタムCSS
custom_css = """
body {
    font-family: Inter, Public Sans, sans-serif;
    font-size: 20px !important;
}

# .header-container {
#     display: flex;
#     align-items: center;
#     margin-bottom: 20px;  /* ヘッダーと説明文の間にスペースを追加 */
# }

.nasa_logo img{
    margin-left: 50px;
    background-color: black;
}

.logo img{
    margin-right: 50px;
}

.header-container h1 {
    font-size: 40px !important;  /* タイトルのフォントサイズを指定 */
    text-align: left;           /* タイトルを中央揃え */
    margin-top: 50px;          /* タイトルの上にスペースを追加 */
}

.markdown h1 {
    font-size: 400px !important;
    margin-top: 10px;             /* デフォルトのマージンを削除 */
    margin-top: 100px;          /* デフォルトのマージンを削除 */
}

.markdown p {
    font-size: 50px !important;  /* 段落のフォントサイズを指定 */
}
"""

# Gradioインターフェースの作成
with gr.Blocks(theme=gr.themes.Default(primary_hue="cyan", secondary_hue="yellow"), css=custom_css) as demo:
    
    # ヘッダーに画像とタイトルを挿入
    with gr.Row():
        # gr.Imageコンポーネントを使用して画像を表示
        gr.Image('./../images/nasa_space_apps.png', elem_classes="nasa_logo", width="150px", height="150px", container=False, show_download_button=False, show_fullscreen_button=False)
        gr.Image('./../images/logo.png', elem_classes="logo", width="150px", height="150px", container=False, show_download_button=False, show_fullscreen_button=False)
        # gr.Markdown("<h1>CO2 Fossil Fuel Emissions Map</h1>", elem_classes="header-container")
    
    # ヘッダーの下に説明文
    gr.Markdown("""
    <h2><p>This map shows the human-caused and natural greenhouse gas emissions data over the years.</p></h2>
    """)
    
    # テキストボックスで都市名を検索
    city_input = gr.Textbox(label="Enter a city to search for GHG emission data:", placeholder="Type city name...", lines=1)
    
    # マップ表示エリア
    output = gr.HTML()
    
    # スライダー（検索バー）を配置
    year_slider = gr.Slider(2001, 2022, step=1, label="Select Year", interactive=True)
    
    city_input.submit(year_map, inputs=[city_input, year_slider], outputs=output)
    # スライダーを動かすとマップを更新
    year_slider.change(year_map, inputs=[city_input, year_slider], outputs=output)

# アプリケーションを起動
demo.launch()

/Users/yuzu/Library/Python/3.9/lib/python/site-packages/gradio/utils.py:1002: UserWarning: Expected 1 arguments for function <function year_map at 0x1755d1af0>, received 2.
  warnings.warn(
/Users/yuzu/Library/Python/3.9/lib/python/site-packages/gradio/utils.py:1010: UserWarning: Expected maximum 1 arguments for function <function year_map at 0x1755d1af0>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/yuzu/Library/Python/3.9/lib/python/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/Users/yuzu/Library/Python/3.9/lib/python/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/yuzu/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/Users/yuzu/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/Users/yuzu/Library/Python/3.9/lib/python/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/yuzu/Library/Python/3.9/lib/python/site-packages/anyio/_backends/_asyncio.py", line 2144, in run_sync_in_worker_thread
    re